# Web Scraping
Según el procedimiento en https://www.datacamp.com/community/tutorials/web-scraping-using-python

In [1]:
#Importes necesarios
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from urllib.request import urlopen
from bs4 import BeautifulSoup
import datetime

In [62]:
#Establecer función para el cambio de fechas
def fecha_variable(start_date):
    fecha = start_date.strftime("%Y-%m-%d")
    url = "https://www.elsiglodetorreon.com.mx/archivo/?d=" + fecha
    html = urlopen(url)
    soup = BeautifulSoup(html, 'lxml')
    for ultag in soup.find_all('ul', {'class': 'list-group'}):
         for litag in ultag.find_all('a'):
                    f = open('%s.csv' % fecha, "a", encoding='utf-8')
                    f.write(litag.text + "\n")
                    f.close()

In [63]:
#Fecha variable

start_date = datetime.date(2020, 2, 1)
end_date = datetime.date(2020, 2, 28)
delta = datetime.timedelta(days=1)

while start_date <= end_date:
    fecha_variable(start_date)
    start_date += delta

### Pendientes: 
- Cuando el titular tiene una coma, produce errores en el CSV  
- Crear dataframe para su uso con Matplotlib  
- Añadir lista de HTML para ingreso a cada una de las notas  